# End of week 2 exercise

Now use everything you've learned from week 2 to build full prototype for the technical question/answerer you built in week 1 Exercise.  
This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

In [12]:
# imports
import os
from dotenv import load_dotenv
from openai import OpenAI
import ollama 
import gradio as gr

In [13]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2:1b'

In [14]:
# set up environment
load_dotenv()
openai = OpenAI()

In [15]:
# system prompt
system_prompt = """
You are a thoughtful and intuitive Book Doctor.
Your job is to recommend books to people based on how they are feeling in the moment.
You respond with warmth, empathy, and insight.

For each recommendation:
- Briefly acknowledge the person's emotional state.
- Recommend 1–3 books.
- Explain why each book fits their current feeling.
- Keep the tone supportive, not clinical.
"""

In [16]:
#core function
def book_doctor(feeling, model_choice):
    
    user_prompt = "Here’s how I’m feeling: " + feeling

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

    # ---------------- GPT (Streaming) ----------------
    if model_choice == "GPT-4o-mini":
        stream = openai.chat.completions.create(
            model=MODEL_GPT,
            messages=messages,
            stream=True
        )

        response = ""
        for chunk in stream:
            content = chunk.choices[0].delta.content
            if content:
                response += content
                yield response

    # ---------------- LLAMA ----------------
    elif model_choice == "Llama 3.2 (Ollama)":
        response = ollama.chat(
            model=MODEL_LLAMA,
            messages=messages
        )
        yield response["message"]["content"]


In [17]:
# Gradio UI
with gr.Blocks(theme=gr.themes.Soft()) as app:
    
    gr.Markdown("# 📚 Book Doctor")
    gr.Markdown("Describe how you're feeling and receive a thoughtful book recommendation.")
    
    feeling_input = gr.Textbox(
        label="How are you feeling?",
        placeholder="I'm feeling stuck and unmotivated lately...",
        lines=4
    )
    
    model_selector = gr.Dropdown(
        choices=["GPT-4o-mini", "Llama 3.2 (Ollama)"],
        value="GPT-4o-mini",
        label="Choose Model"
    )
    
    output = gr.Markdown()
    
    submit_btn = gr.Button("Get Recommendation")

    submit_btn.click(
        fn=book_doctor,
        inputs=[feeling_input, model_selector],
        outputs=output
    )

app.launch()


/var/folders/g5/vwwf_95j4wsbr81dryy15pdc0000gn/T/ipykernel_99097/727154472.py:2: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Soft()) as app:


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
